In [ ]:
import pickle

In [ ]:
#open file of gif database 
#create array with each element containing url and description of a gif
f = open("gif.pkl","rb")
gif_list = pickle.load(f)

[['https://38.media.tumblr.com/9f6c25cc350f12aa74a7dc386a5c4985/tumblr_mevmyaKtDf1rgvhr8o1_500.gif', 'a man is glaring, and someone with sunglasses appears.'], ['https://38.media.tumblr.com/9ead028ef62004ef6ac2b92e52edd210/tumblr_nok4eeONTv1s2yegdo1_400.gif', 'a cat tries to catch a mouse on a tablet'], ['https://38.media.tumblr.com/9f43dc410be85b1159d1f42663d811d7/tumblr_mllh01J96X1s9npefo1_250.gif', 'a man dressed in red is dancing.'], ['https://38.media.tumblr.com/9f659499c8754e40cf3f7ac21d08dae6/tumblr_nqlr0rn8ox1r2r0koo1_400.gif', 'an animal comes close to another in the jungle'], ['https://38.media.tumblr.com/9ed1c99afa7d71411884101cb054f35f/tumblr_mvtuwlhSkE1qbnleeo1_500.gif', 'a man in a hat adjusts his tie and makes a weird face.'], ['https://38.media.tumblr.com/9e437d26769cb2ac4217df14dbb20034/tumblr_npw7v7W07C1tmj047o1_250.gif', 'someone puts a cat on wrapping paper then wraps it up and puts on a bow'], ['https://38.media.tumblr.com/9e4ab65c0e7d4bb8aa6b5be854b83794/tumblr_md

In [ ]:
pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 50.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 16.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 5.1 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 5.7 MB/s eta 0:00:000:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 57.9 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 62.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 76.5 MB/s eta 0:00:0000:0100:01
     ━

In [ ]:
from sentence_transformers import SentenceTransformer

# Initialize retriever with SentenceTransformer model 
# Retrievers (NLP transformer model) are fine-tuned for creating sentence embeddings/vectors which represent meaning of text
retriever = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
print (retriever)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)


In [ ]:
import numpy as np

In [ ]:
# gif_list needed as array for functionality with numpy
# gif_array = np.array(gif_list)[:,1]

In [ ]:
# tqdm visually shows progress of operation as percentage and time remaining until completion
from tqdm.auto import tqdm

# we will use batches of 1 - how many gifs are processed at a time?
batch_size = 1
# array created so data can be appended as for loop progresses
meta = []

for i in tqdm(range(0, len(gif_list), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(gif_list))
    # extract batch
    batch = gif_list[i:i_end]
    # isolate description of each gif
    gif_array = np.array(batch)[:,1]
    # generate embeddings for batch
    emb = retriever.encode(gif_array).tolist()
    # get metadata
    meta.append({'url': gif_list[i][0], 'description': gif_list[i][1], 'emb':emb[0]})
    
    
# saves data for each gif, including its vector transform, in a file 
f = open('embedded_gif.pkl', "wb")
pickle.dump(meta,f)

  0%|          | 0/125782 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# retrieve gif data from file including url, description and vector transform
f = open ("embedded_gif.pkl", "rb")
emb_gif = pickle.load(f)

In [ ]:
response = ""
def embed(response):
    emb_response = retriever.encode(response).tolist()
    list_cos_sim = []
    for gif in emb_gif:
        cos_sim = abs(np.dot (gif['emb'],emb_response))
        list_cos_sim.append(cos_sim)
    list_cos_sim = np.array(list_cos_sim)
    max_idx = np.argmax(list_cos_sim)
    
    print (emb_gif[max_idx]['url'], emb_gif[max_idx]['description'])

In [ ]:
# predicted response generated from conversatoin and fed into gif generator
response1 = "Thanks for the suggestions. I think I'll go for a walk in Golden Gate Park."
embed(response1)

https://33.media.tumblr.com/5232742ce62600ce6ae642a9cdeb4d57/tumblr_npph43K5cy1uy3nwqo1_500.gif a large gate is opening up to a large park


In [ ]:
response2 = "I am having a bowl of cereal."
embed(response2)

https://33.media.tumblr.com/a05194b9759accfcb23cf6523703bf88/tumblr_nfuskq9AxT1s6zpepo1_250.gif a man is going to eat a bowl of cereal.


In [ ]:
response3 = "I'm not sure. I just feel like I need to talk to someone."
embed(response3)

https://31.media.tumblr.com/6c8f5d92dd773a1e56d61a23595da8a7/tumblr_ngut3pJE2T1u6n2x3o1_250.gif a woman is sitting down and talking to somebody
